## Climate Model and Climate Data Workshop - Hands-on Session

This Jupyter notebook showcases how to use climate model data in python. However, it is not a Python course!

We will use the xarray package: https://docs.xarray.dev/en/stable/index.html

## Exercise 1: Loading and Inspecting Data

First, packages which we will use later need to be loaded.

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
# read cmip6 data from netcdf
inpath='/net/co2/c2sm-data/rlorenz/climate_model_data_workshop'
ifile = f'{inpath}/tas_Amon_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_201001-201412.nc'
variable_name='tas'
with xr.open_dataset(ifile) as ds:
    da = ds[variable_name]
print(da)

In [ ]:
# read data for one year only
with xr.open_dataset(ifile).sel(time='2012') as ds:
    da_tim = ds[variable_name]
print(da_tim)

## Exercise 2: Subsetting Data

In [ ]:
# read data but only for polygon
min_lon =  5. 
max_lon = 16. 
# latitude is -90 to 90
min_lat = 45.
max_lat = 55.
with xr.open_dataset(ifile).sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon)) as ds:
    da_reg = ds[variable_name]
print(da_reg)

In [ ]:
# extract certain region using regionmask package, 
import regionmask
print(regionmask.defined_regions.srex)

In [ ]:
lon=da['lon']
lat=da['lat']

In [ ]:
mask = regionmask.defined_regions.srex.mask(lon, lat)
CEU_index = regionmask.defined_regions.srex.map_keys("C. Europe")
print(f'CEU_index: {CEU_index}')
mask_CEU = mask == CEU_index
da_CEU = da.isel(time=1).where(mask_CEU)
print(da_CEU)

## Exercise 3: Plotting and Visualization

### 3.1 Regional Maps

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
# choose a good projection for regional maps
proj=ccrs.LambertConformal(central_longitude=15)

# plot the outline of the central European region
ax = regionmask.defined_regions.srex.plot(add_ocean=False, resolution='50m',
                          projection=proj, add_label=True)

da_CEU.plot.pcolormesh('lon', 'lat', ax=ax, transform=ccrs.PlateCarree())

ax.coastlines(linewidth=0.5)

# fine tune the extent
ax.set_extent([-15, 45, 40, 65], crs=ccrs.PlateCarree())

In [ ]:
CH = regionmask.defined_regions.natural_earth_v5_0_0.countries_110[["Switzerland"]]
mask = CH.mask(da.lon, da.lat)
da_CH = da.isel(time=1).where(mask==127)

In [ ]:
# choose a good projection for regional maps
proj=ccrs.LambertConformal(central_longitude=15)
ax = plt.subplot(111, projection=proj)

da_CH.plot.pcolormesh(ax=ax, x="lon", y="lat", transform=ccrs.PlateCarree())
regionmask.defined_regions.natural_earth_v5_0_0.countries_110[["Switzerland"]].plot(ax=ax, add_label=False)

ax.coastlines(linewidth=0.5)

# fine tune the extent
ax.set_extent([-15, 45, 40, 65], crs=ccrs.PlateCarree())

In [ ]:
# read data and calculate seasonal means
with xr.open_dataset(ifile) as ds:
    ds_seas = ds.groupby("time.season").mean(dim='time')
print(ds_seas)

### 3.2 Global Maps

In [ ]:
# choose a good projection for global maps
proj = ccrs.Robinson()

ax = plt.subplot(111, projection=proj)

ds_seas.sel(season='JJA').tas.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree())

ax.coastlines();

There are many other things xarray can do, have a look for instance here: 
- https://docs.xarray.dev/en/latest/howdoi.html
- https://docs.xarray.dev/en/stable/index.html